# Aggregate Data
Your task is to aggregate the data to showcase obvious trends in school performance.

## 0. Setup

**- Import dependencies.**

In [1]:
import pandas as pd

**- Access schools and students data in CSV files and store into DataFrames.**

In [2]:
# Store file path strings into variables.
sch_fp = 'Resources/schools_complete.csv'
stu_fp = 'Resources/students_complete.csv'

# Store schools and students data from CSVs into DataFrames.
sch_df = pd.read_csv(sch_fp, delimiter=',')
stu_df = pd.read_csv(stu_fp, delimiter=',')

**- Rename columns in the Schools and Students DataFrames.** 

In [3]:
sch_df = sch_df.rename(
    columns = {
        'school_name': 'School Name', 'type': 'Type', 'size': 'Size', 'budget': 'Budget'
    }
)
stu_df = stu_df.rename(
    columns = {
        'student_name': 'Student Name', 'gender': 'Gender', 'grade': 'Grade', 'budget': 'Budget', 
        'school_name': 'School Name', 'reading_score': 'Reading Score', 'math_score': 'Math Score'
    }
)

## 1. District Summary
Create a high-level snapshot, in a DataFrame, of the district's key metrics, including the following:

* Total schools
* Total students
* Total budget
* Average math score
* Average reading score
* % passing math (the percentage of students who passed math)
* % passing reading (the percentage of students who passed reading)
* % overall passing (the percentage of students who passed math AND reading)

**- Create DataFrames that filter the Student DataFrame based on passing criteria.**
- Referred to for passing metrics.

In [13]:
# Students Who Passed Math (i.e., 70 or greater under "Math Score")
pass_math_df = stu_df.loc[stu_df['Math Score'] >= 70]

# Students Who Passed Reading (i.e., 70 or greater under "Reading Score")
pass_read_df = stu_df.loc[stu_df['Reading Score'] >= 70]

# Students Who Passed Math and Reading (i.e., 70 or greater under "Math Score" and "Reading Score")
pass_both_df = stu_df.loc[(stu_df['Math Score'] >= 70) & (stu_df['Reading Score'] >= 70)]

**- Calculate district's key metrics and store in variables.**

In [5]:
# Total Schools
sch_cnt = len(sch_df)

# Total Students
stu_cnt = len(stu_df)

# Total Budget
bgt_ttl = sch_df['Budget'].sum()

# Average Math Score
math_avg = stu_df['Math Score'].mean()

# Average Reading Score
read_avg = stu_df['Reading Score'].mean()

# % Passing Math (i.e., the percentage of students who passed math)
pass_math_pct = (len(pass_math_df) / stu_cnt) * 100

# % Passing Reading (i.e., the percentage of students who passed reading)
pass_read_pct = (len(pass_read_df) / stu_cnt) * 100

# % Overall Passing (i.e., the percentage of students who passed math and reading)
pass_both_pct = (len(pass_both_df) / stu_cnt) * 100

**- Create the District Summary DataFrame with variables containing key metrics for high-level snapshot of the district.**

In [81]:
dist_summ_df = pd.DataFrame([
    {
        'Total Schools': sch_cnt,
        'Total Students': stu_cnt,
        'Total Budget': bgt_ttl,
        'Average Math Score': math_avg,
        'Average Reading Score': read_avg,
        '% Passing Math': pass_math_pct,
        '% Passing Reading': pass_read_pct,
        '% Overall Passing': pass_both_pct
    }
])

**- Create copy of the District Summary DataFrame to separate formatting transformations from underlying data.**

In [87]:
dist_summ_disp_df = dist_summ_df.copy()

**- Apply formatting to the metrics in the copied District Summary DataFrame.**

In [88]:
dist_summ_disp_df['Total Schools'] = dist_summ_df['Total Schools'].map("{:,}".format)
dist_summ_disp_df['Total Students'] = dist_summ_df['Total Students'].map("{:,}".format)
dist_summ_disp_df['Total Budget'] = dist_summ_df['Total Budget'].map("${:,.2f}".format)
dist_summ_disp_df['% Passing Math'] = dist_summ_df['% Passing Math'].map("{:.2f}%".format)
dist_summ_disp_df['% Passing Reading'] = dist_summ_df['% Passing Reading'].map("{:.2f}%".format)
dist_summ_disp_df['% Overall Passing'] = dist_summ_df['% Overall Passing'].map("{:.2f}%".format)

**- Display the copied District Summary DataFrame.**

In [89]:
dist_summ_disp_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.98%,85.81%,65.17%


## 2. School Summary

Create a DataFrame that summarizes key metrics about each school, including the following:

* School name
* School type
* Total students
* Total school budget
* Per student budget
* Average math score
* Average reading score
* % passing math (the percentage of students who passed math)
* % passing reading (the percentage of students who passed reading)
* % overall passing (the percentage of students who passed math AND reading)

**- Merge the Schools and Students DataFrames in a new DataFrame (Students with Schools), adding Schools data for each student.**

In [10]:
stu_with_sch_data_df = pd.merge(stu_df, sch_df, how='left', on='School Name')

**- Create DataFrameGroupBy objects by grouping DataFrames by "School Name".**
* "School Name" will be the index of subsequent DataFrames created from these grouped DataFrames.

In [18]:
# From the "Students with Schools" DataFrame
sch_grp = stu_with_sch_data_df.groupby(['School Name'])

# From the "Students Who Passed Math" DataFrame
pass_math_grp = pass_math_df.groupby(['School Name'])

# From the "Students Who Passed Reading" DataFrame
pass_read_grp = pass_read_df.groupby(['School Name'])

# From the "Students Who Passed Math and Reading" DataFrame
pass_both_grp = pass_both_df.groupby(['School Name'])

**- Calculate metrics by "School Name" and store into variables.**
* Stored as Series.

In [20]:
# School Type
sch_type = sch_grp['Type'].min()

# Total Students
stu_cnt_by_sch = sch_grp['Student ID'].count()

# Total Budget
bgt_ttl_by_sch = sch_grp['Budget'].min()

# Per Student Budget
per_stu_bgt_by_sch = bgt_ttl_by_sch / stu_cnt_by_sch

# Average Math Score
math_avg_by_sch = sch_grp['Math Score'].mean()

# Average Reading Score
read_avg_by_sch = sch_grp['Reading Score'].mean()

# % of Students Who Passed Math
pass_math_pct_by_sch = (pass_math_grp['Student ID'].count() / stu_cnt_by_sch) * 100

# % of Students Who Passed Reading
pass_read_pct_by_sch = (pass_read_grp['Student ID'].count() / stu_cnt_by_sch) * 100

# % of Students Who Passed Math and Reading
pass_both_pct_by_sch = (pass_both_grp['Student ID'].count() / stu_cnt_by_sch) * 100

**- Create the School Summary DataFrame with variables containing key metrics by "School Name".**

In [92]:
sch_summ_df = pd.DataFrame(
    {
        'School Type': sch_type,
        'Total Students': stu_cnt_by_sch,
        'Total School Budget': bgt_ttl_by_sch,
        'Per Student Budget': per_stu_bgt_by_sch,
        'Average Math Score': math_avg_by_sch,
        'Average Reading Score': read_avg_by_sch,
        '% Passing Math': pass_math_pct_by_sch,
        '% Passing Reading': pass_read_pct_by_sch,
        '% Overall Passing': pass_both_pct_by_sch
    }
)

**- Create copy of the School Summary DataFrame to separate formatting transformations from underlying data.**

In [93]:
sch_summ_disp_df = sch_summ_df.copy()

**- Apply formatting to the metrics in the School Summary DataFrame.**

In [94]:
sch_summ_disp_df['Total Students'] = sch_summ_df['Total Students'].map("{:,}".format)
sch_summ_disp_df['Total School Budget'] = sch_summ_df['Total School Budget'].map("${:,.2f}".format)
sch_summ_disp_df['Per Student Budget'] = sch_summ_df['Per Student Budget'].map("${:,.2f}".format)
sch_summ_disp_df['% Passing Math'] = sch_summ_df['% Passing Math'].map("{:.2f}%".format)
sch_summ_disp_df['% Passing Reading'] = sch_summ_df['% Passing Reading'].map("{:.2f}%".format)
sch_summ_disp_df['% Overall Passing'] = sch_summ_df['% Overall Passing'].map("{:.2f}%".format)

**- Display the School Summary DataFrame.**

In [95]:
sch_summ_disp_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.68%,81.93%,54.64%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.13%,97.04%,91.33%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.99%,80.74%,53.20%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.31%,79.30%,54.29%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.39%,97.14%,90.60%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.51%,96.25%,89.23%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.68%,81.32%,53.51%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.06%,81.22%,53.54%


## 3. Highest-Performing Schools (by % Overall Passing)

Create a DataFrame that highlights the top 5 performing schools based on % Overall Passing. Include the following metrics:

* School name
* School type
* Total students
* Total school budget
* Per student budget
* Average math score
* Average reading score
* % passing math (the percentage of students who passed math)
* % passing reading (the percentage of students who passed reading)
* % overall passing (the percentage of students who passed math AND reading)

**- Sort the School Summary DataFrame by % Overall Passing (descending), show the first 5 rows, and save into a new DataFrame (Highest-Performing Schools).**
- Applied sort on the base School Summary DataFrame to ensure sort is based on values, not strings.

In [100]:
sch_top5_df = sch_summ_df.sort_values('% Overall Passing', ascending = False).head(5)

**- Apply formatting to the metrics in the Highest-Performing Schools DataFrame.**

In [102]:
sch_top5_df['Total Students'] = sch_top5_df['Total Students'].map("{:,}".format)
sch_top5_df['Total School Budget'] = sch_top5_df['Total School Budget'].map("${:,.2f}".format)
sch_top5_df['Per Student Budget'] = sch_top5_df['Per Student Budget'].map("${:,.2f}".format)
sch_top5_df['% Passing Math'] = sch_top5_df['% Passing Math'].map("{:.2f}%".format)
sch_top5_df['% Passing Reading'] = sch_top5_df['% Passing Reading'].map("{:.2f}%".format)
sch_top5_df['% Overall Passing'] = sch_top5_df['% Overall Passing'].map("{:.2f}%".format)

**- Display the Highest-Performing Schools DataFrame.**

In [103]:
sch_top5_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.59%,95.95%,90.54%


## 4. Lowest-Performing Schools (by % Overall Passing)

Create a DataFrame that highlights the bottom 5 performing schools based on % Overall Passing. Include the following metrics:

* School name
* School type
* Total students
* Total school budget
* Per student budget
* Average math score
* Average reading score
* % passing math (the percentage of students who passed math)
* % passing reading (the percentage of students who passed reading)
* % overall passing (the percentage of students who passed math AND reading)

**- Sort the School Summary DataFrame by % Overall Passing (ascending), show the first 5 rows, and save into a new DataFrame (Lowest-Performing Schools).***
- Applied sort on the base School Summary DataFrame to ensure sort is based on values, not strings.

In [105]:
sch_bot5_df = sch_summ_df.sort_values('% Overall Passing', ascending = True).head(5)

**- Apply formatting to the metrics in the Lowest-Performing Schools DataFrame.**

In [106]:
sch_bot5_df['Total Students'] = sch_bot5_df['Total Students'].map("{:,}".format)
sch_bot5_df['Total School Budget'] = sch_bot5_df['Total School Budget'].map("${:,.2f}".format)
sch_bot5_df['Per Student Budget'] = sch_bot5_df['Per Student Budget'].map("${:,.2f}".format)
sch_bot5_df['% Passing Math'] = sch_bot5_df['% Passing Math'].map("{:.2f}%".format)
sch_bot5_df['% Passing Reading'] = sch_bot5_df['% Passing Reading'].map("{:.2f}%".format)
sch_bot5_df['% Overall Passing'] = sch_bot5_df['% Overall Passing'].map("{:.2f}%".format)

**- Display the Lowest-Performing Schools DataFrame.**

In [107]:
sch_bot5_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.37%,80.22%,52.99%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.99%,80.74%,53.20%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.68%,81.32%,53.51%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.75%,80.86%,53.53%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.06%,81.22%,53.54%


## 5. Math Scores by Grade

Create a DataFrame that lists the average math score for students of each grade level (9th, 10th, 11th, 12th) at each school.

**- Create DataFrames that filter the Student DataFrame based on grade criteria.**

In [51]:
stu_09th_df = stu_df.loc[stu_df['Grade'] == '9th', :]
stu_10th_df = stu_df.loc[stu_df['Grade'] == '10th', :]
stu_11th_df = stu_df.loc[stu_df['Grade'] == '11th', :]
stu_12th_df = stu_df.loc[stu_df['Grade'] == '12th', :]

**- Create DataFrameGroupBy objects for each of the grade-specific DataFrames by grouping by "School Name".**
* "School Name" will be the index of subsequent DataFrames created from these grouped DataFrames.

In [52]:
sch_09th_grp = stu_09th_df.groupby('School Name')
sch_10th_grp = stu_10th_df.groupby('School Name')
sch_11th_grp = stu_11th_df.groupby('School Name')
sch_12th_grp = stu_12th_df.groupby('School Name')

**- Calculate grade-specific math metrics by "School Name" and store into variables.**
* Stored as Series.

In [55]:
# 9th Grade Math Score Average
math_avg_09th_by_sch = sch_09th_grp['Math Score'].mean()

# 10th Grade Math Score Average
math_avg_10th_by_sch = sch_10th_grp['Math Score'].mean()

# 11th Grade Math Score Average
math_avg_11th_by_sch = sch_11th_grp['Math Score'].mean()

# 12th Grade Math Score Average
math_avg_12th_by_sch = sch_12th_grp['Math Score'].mean()

**- Create the Math Scores by Grade DataFrame with variables containing grade-specific math metrics by "School Name".**

In [68]:
avg_math_by_sch_grd_df = pd.DataFrame(
    {
        '9th': math_avg_09th_by_sch, 
        '10th': math_avg_10th_by_sch, 
        '11th': math_avg_11th_by_sch, 
        '12th': math_avg_12th_by_sch
    }
)

**- Display the Math Scores by Grade DataFrame.**

In [69]:
avg_math_by_sch_grd_df

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## 6. Reading Scores by Grade

Create a DataFrame that lists the average reading score for students of each grade level (9th, 10th, 11th, 12th) at each school.

**- Calculate grade-specific reading metrics by "School Name" and store into variables.**
* Stored as Series.

In [70]:
# 9th Grade Reading Score Average
read_avg_09th_by_sch = sch_09th_grp['Reading Score'].mean()

# 10th Grade Math Score Average
read_avg_10th_by_sch = sch_10th_grp['Reading Score'].mean()

# 11th Grade Math Score Average
read_avg_11th_by_sch = sch_11th_grp['Reading Score'].mean()

# 12th Grade Math Score Average
read_avg_12th_by_sch = sch_12th_grp['Reading Score'].mean()

**- Create the Reading Scores by Grade DataFrame with variables containing grade-specific reading metrics by "School Name".**

In [71]:
avg_read_by_sch_grd_df = pd.DataFrame(
    {
        '9th': read_avg_09th_by_sch, 
        '10th': read_avg_10th_by_sch, 
        '11th': read_avg_11th_by_sch, 
        '12th': read_avg_12th_by_sch
    }
)

**- Display the Reading Scores by Grade DataFrame.**

In [72]:
avg_read_by_sch_grd_df

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## 7. Scores by School Spending

Create a table that breaks down school performance based on average spending ranges (per student). Use your judgment to create four bins with reasonable cutoff values to group school spending. Include the following metrics in the table:

* Average math score
* Average reading score
* % passing math (the percentage of students who passed math)
* % passing reading (the percentage of students who passed reading)
* % overall passing (the percentage of students who passed math AND reading)

**- Set the 4 bins for "Per Student Budget".**

In [118]:
per_stu_bgt_bin = [0, 585, 630, 645, 680]
per_stu_bgt_bin_nms = ["Less Than 585", "585 to 630", "630 to 645", "More Than 645"]

**- Add a new Series ("Per Student Budget Ranges") to the Student Summary DataFrame using the determined list of bins.**

In [119]:
sch_summ_df['Per Student Budget Ranges'] = pd.cut(
    sch_summ_df['Per Student Budget'], 
    per_stu_bgt_bin, 
    labels = per_stu_bgt_bin_nms
)

**- Create a DataFrameGroupBy object from the School Summary DataFrame by grouping by "Per Student Budget Ranges".**

In [121]:
per_stu_bgt_grp = sch_summ_df.groupby('Per Student Budget Ranges')

**- Create the Scores by School Spending DataFrame by taking the mean on the School Summary DataFrame metrics.**

In [127]:
sc_by_bgt_df = per_stu_bgt_grp[[
    'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing'
]].mean()

**- Apply formatting to the metrics in the Scores by School Spending DataFrame.**

In [129]:
sc_by_bgt_df['% Passing Math'] = sc_by_bgt_df['% Passing Math'].map("{:.2f}%".format)
sc_by_bgt_df['% Passing Reading'] = sc_by_bgt_df['% Passing Reading'].map("{:.2f}%".format)
sc_by_bgt_df['% Overall Passing'] = sc_by_bgt_df['% Overall Passing'].map("{:.2f}%".format)

**- Display the Scores by School Spending DataFrame.**

In [130]:
sc_by_bgt_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Per Student Budget Ranges,,,,,
Less Than 585,83.455399,83.933814,93.46%,96.61%,90.37%
585 to 630,81.899826,83.155286,87.13%,92.72%,81.42%
630 to 645,78.518855,81.624473,73.48%,84.39%,62.86%
More Than 645,76.997210,81.027843,66.16%,81.13%,53.53%


## 8. Scores by School Size

Create a table that breaks down school performance based on school size (small, medium, or large).

**- Set the bins for "School Size".**

In [133]:
sch_size_bin = [0, 1000, 2000, 5000]
sch_size_bin_nms = ["Small (Less Than 1000)", "Medium (1000 to 2000)", "Large (More Than 2000)"]

**- Add a new Series ("School Size") to the Student Summary DataFrame using the determined list of bins.**

In [134]:
sch_summ_df['School Size'] = pd.cut(
    sch_summ_df['Total Students'], 
    sch_size_bin, 
    labels = sch_size_bin_nms
)

**- Create a DataFrameGroupBy object from the School Summary DataFrame by grouping by "School Size".**

In [135]:
sch_size_grp = sch_summ_df.groupby('School Size')

**- Create the Scores by School Size DataFrame by taking the mean on the School Summary DataFrame metrics.**

In [136]:
sc_by_sch_size_df = sch_size_grp[[
    'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing'
]].mean()

**- Apply formatting to the metrics in the Scores by School Size DataFrame.**

In [137]:
sc_by_sch_size_df['% Passing Math'] = sc_by_sch_size_df['% Passing Math'].map("{:.2f}%".format)
sc_by_sch_size_df['% Passing Reading'] = sc_by_sch_size_df['% Passing Reading'].map("{:.2f}%".format)
sc_by_sch_size_df['% Overall Passing'] = sc_by_sch_size_df['% Overall Passing'].map("{:.2f}%".format)

**- Display the Scores by School Spending DataFrame.**

In [138]:
sc_by_sch_size_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (Less Than 1000),83.821598,83.929843,93.55%,96.10%,89.88%
Medium (1000 to 2000),83.374684,83.864438,93.60%,96.79%,90.62%
Large (More Than 2000),77.746417,81.344493,69.96%,82.77%,58.29%


## 9. Scores by School Type

Create a table that breaks down school performance based on type of school (district or charter).

**- Create a DataFrameGroupBy object from the School Summary DataFrame by grouping by "Type".**

In [140]:
sch_type_grp = sch_summ_df.groupby('School Type')

**- Create the Scores by School Type DataFrame by taking the mean on the School Summary DataFrame metrics.**

In [141]:
sc_by_sch_type_df = sch_type_grp[[
    'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing'
]].mean()

**- Apply formatting to the metrics in the Scores by School Type DataFrame.**

In [142]:
sc_by_sch_type_df['% Passing Math'] = sc_by_sch_type_df['% Passing Math'].map("{:.2f}%".format)
sc_by_sch_type_df['% Passing Reading'] = sc_by_sch_type_df['% Passing Reading'].map("{:.2f}%".format)
sc_by_sch_type_df['% Overall Passing'] = sc_by_sch_type_df['% Overall Passing'].map("{:.2f}%".format)

**- Display the Scores by School Type DataFrame.**

In [143]:
sc_by_sch_type_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.62%,96.59%,90.43%
District,76.956733,80.966636,66.55%,80.80%,53.67%


#  Analysis on School Performance
My observations on aggregated data:

1. As a whole, the district's school performance is poor: Despite 74.98% of students passing math and 85.81% of students passing reading, only **65.17%** is passing both math and reading (i.e. overall), which is below the 70% requirement as passing.
2. The top 5 schools in overall passing % are all charter schools; in contrast, the bottom 5 schools in overall passing % are all district schools. Overall, **90.43%** of charter schools' students passed both math and reading; in contrast, only **53.67%** of district schools' students passed both math and reading.
    - Note that charter schools have less students than district schools. However, this doesn't imply a higher budget per student; in fact, charter schools have a lower budget per student than district schools (\\$578 to \\$638 for charter schools versus \\$622 to \\$655 for district schools). 
        - Thus, the per student budget amount is not indicative of school performance (90.37% versus 53.53% overall passing between schools with a per student budget of less than \\$585 versus a per student budget of more than \\$645).
        - Only 58.29% of students in the eight schools that have more than 2,000 students passed both math and reading. Only one of these schools was a charter school (Wilson High School), with 90.58% of its students passing both math and reading. 
    - The difference in percentage of students passing math between charter and district schools is much starker than that of students passing reading (93.62% of charter school students versus 66.55% of district students).

**Therefore, the district's overall school performance can be best explained by its stark dichotomy between its charter and district schools.**